In [1]:
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma

In [2]:

from dotenv import load_dotenv
load_dotenv() 

True

In [3]:
pdf_path = Path("./nodejs.pdf")

In [4]:

load = PyPDFLoader(file_path = pdf_path)
doc = load.load()


Ignoring wrong pointing object 268 0 (offset 0)
Ignoring wrong pointing object 309 0 (offset 0)


In [5]:

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
)

In [6]:
spited = text_splitter.split_documents(documents=doc)

In [9]:
import os
if not os.path.exists("qdrant_store"):
    store = Chroma.from_documents(
    documents=spited,
    embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"),
    persist_directory="qdrant_store"
    )
    print("New Chroma DB created.")
else:
    print("Directory already exists. Skipping creation.")

New Chroma DB created.


In [10]:
retriever = store.as_retriever(
     search_type = "similarity",
        search_kwargs = {
            "k":10 
        }
)

In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.3, max_tokens=500)

In [18]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains import LLMChain


In [13]:
system_prompt = (
    "You are a helpful assistant that answers questions about Data Science {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{input}"),
    ]
)

In [14]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)


In [17]:

query_variation_prompt = PromptTemplate.from_template(
    "Generate 5 diverse and useful variations of the following question for better document retrieval:\n\nQuestion: {question}\n\nVariations:"
)

In [19]:
variation_chain = LLMChain(llm=llm, prompt=query_variation_prompt)

C:\Users\yashp\AppData\Local\Temp\ipykernel_12772\1357315315.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  variation_chain = LLMChain(llm=llm, prompt=query_variation_prompt)


In [21]:
user_question = "Can you explain routing in nodejs briefly?"

In [22]:
variation_response = variation_chain.invoke({"question": user_question})
query_variations = [line.strip("- ").strip() for line in variation_response['text'].split("\n") if line.strip()]
all_queries = [user_question] + query_variations

In [28]:
print("\n🧠 Generated Query Variations:")
for i, q in enumerate(query_variations, 1):
    print(f"{q}")


🧠 Generated Query Variations:
1. **Node.js Routing: A concise explanation** (Focuses on brevity and uses a keyword likely found in documentation titles)
2. **How are routes defined and handled in a Node.js application?** (More specific about the mechanics of routing)
3. **What are the core concepts and components involved in Node.js routing (e.g., request, response, middleware)?** (Emphasizes key elements and encourages structured explanations)
4. **Best practices for implementing efficient and scalable routing in Node.js** (Shifts focus towards practical application and optimization)
5. **Comparison of different Node.js routing libraries/frameworks (e.g., Express.js, Hapi.js).** (Expands the scope to include common tools and encourages comparative analysis)


In [26]:
retrieved_docs = retriever.invoke(user_question)

response = question_answer_chain.invoke({
    "context": retrieved_docs,
    "input": user_question
})

print("\n📘 Final Answer:\n", response)


📘 Final Answer:
 Routing in Node.js, typically using the Express.js framework, directs incoming HTTP requests to appropriate handler functions based on the URL path.  Think of it as a traffic controller for your web application.

Here's a breakdown:

1. **Request:** A client (like a web browser) sends an HTTP request to your server with a specific URL (e.g., `/users` or `/products/123`).

2. **Router:** Express.js uses a `Router` object to match the request's URL path to predefined routes.

3. **Routes:** Routes are defined using HTTP methods (GET, POST, PUT, DELETE, etc.) and URL patterns. For example:

   ```javascript
   const express = require('express');
   const router = express.Router();

   router.get('/users', (req, res) => { 
       // Handle GET request to /users
   });

   router.post('/products', (req, res) => {
       // Handle POST request to /products
   });

   module.exports = router;
   ```

4. **Middleware:** Functions (called middleware) can be executed before the

In [ ]:
response = rag_chain.invoke({"input": "Can you explain Data Frame in Pandas briefly?"})
print(response["answer"])